### Create new price list
(С всеми новыми тарифами в Combined, за исключением новых стран.)

!!! Доработать для добавления новых стран !!!

In [1]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = r'c:\Users\balob\Downloads\DATA'
file_result = 'df_combined_with_new.csv'
file_al = 'DMI.dmi_allowed_list_export_Wed_Apr_10_2019.csv'
file_tagid_list_gsma = 'TADIG_LIST_GSMA_190227.csv'
file_tagid_list_mccmnc_ir21 = 'tadig_mccmnc_ir21.csv'

eur_usd = 1.2

In [2]:
# Из базы данных OCS выгружаем TADIG с NETWORK_ID

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID,MCCMNC FROM
(
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID
,np.MCC + IIF(LEN(MNC)=1,'0' + MNC,MNC) MCCMNC
,ROW_NUMBER() OVER(PARTITION BY c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE ORDER BY t.TADIG_CODE DESC) AS ROW_ID
FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES np ON t.network_id = np.network_id
) t
WHERE ROW_ID=1
'''
df_tadig_mccmnc = pd.read_sql_query(sql_srt, cnxn)
df_tadig = df_tadig_mccmnc[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','NETWORK_ID']].drop_duplicates()

df_tadig_mccmnc = df_tadig_mccmnc[['NETWORK_ID','MCCMNC']].drop_duplicates()

display(df_tadig.head())

cnxn.close()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,4.0
1,Afghanistan,Areeba/MTN,AFGAR,5.0
2,Afghanistan,Etisalat,AFG55,6.0
3,Afghanistan,Etisalat,AFGEA,6.0
4,Afghanistan,Roshan/TDCA,AFGK9,7.0


In [18]:
###############################################
# Опция №1
###############################################

# Загружаем новые тарифы Partner и P4 из файлов

'''df_partner = pd.read_csv(join(load_folder,'Partner_1_1_2019_for_tariffs.csv'),sep=';')
print('Тарифы Partner:')
display(df_partner.head())'''

'''df_p4 = pd.read_csv(join(load_folder,'P4_1_1_2019_for_tariffs.csv'),sep=';')
#df_p4[['MOC','MTC','SMS','MB']] = df_p4[['MOC','MTC','SMS','MB']]*eur_usd
print('Тарифы P4:')
display(df_p4.head())'''

df_mb = pd.read_csv(join(load_folder,'NEW_TARIFFS.csv'),sep=';')
print('Тарифы MB:')
display(df_mb.head())

Тарифы MB:


,Country,Partner Name,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC_S1,Data_Increment_KB_S1
0,HK,3HK,HKGH3,0.013,0.013,0.13,0.003150,1,1
1,Macau,3 Macau,MACHT,0.130,0.130,0.13,0.005040,1,1
2,CHINA,UNICOM,CHNCU,0.260,0.260,0.26,0.006301,1,1
3,UAE,Du,AREDU,0.260,0.260,0.26,0.011971,1,1
4,USA,AT&T,USACG,0.260,0.260,0.26,0.005040,1,1


In [4]:
###############################################
# Опция №2
###############################################

# Загружаем тарифы Partner и P4 из базы данных OCS

# Указать в запросоах OCS roaming_plan_id
# Указать в запросоах RoamDB tarifflistid

#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
#OCS DB Partner cost (roaming_plan_id=267)
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,rpr.mo_call_rate AS MOC
,rpr.mt_call_rate AS MTC
,rpr.mo_sms_rate AS SMS
,rpr.mt_sms_rate
,rpr.data_rate AS MB
,CASE WHEN rpr.currency_id=1 THEN 'USD' WHEN rpr.currency_id=2 THEN 'EUR' END AS Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' END AS Sponsor
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=267 --Partner
AND rpr.end_date IS NULL
AND rpr.is_discounted=1
'''

df_partner_cost = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

# display(df_partner_cost.head())

#Connect to RoamDB
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
#RoamDB Partner cost (tarifflistid=28)
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 28 --'Partner Mobile Costs 2019'
'''
df_partner_2019 = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

# display(df_partner_2019.head())

df_partner=pd.merge(df_partner_cost, df_partner_2019, how='left',left_on='TADIG',right_on='value')

display(df_partner.head(3))

cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')
#OCS DB P4 Cost (roaming_plan_id=268) with convertion to USD from EUR
sql_srt=\
'''
SELECT
c.country_name AS COUNTRY
,n.network_name AS Operator
,td.tadig_code AS TADIG
,mn.mcc + CASE WHEN LEN(mn.mnc)=1 THEN '0'+ mn.mnc ELSE mn.mnc END AS MCCMNC
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_call_rate WHEN rpr.currency_id=2 THEN rpr.mo_call_rate*1.2 END MOC
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_call_rate WHEN rpr.currency_id=2 THEN rpr.mt_call_rate*1.2 END MTC
,CASE WHEN rpr.currency_id=1 THEN rpr.mo_sms_rate WHEN rpr.currency_id=2 THEN rpr.mo_sms_rate*1.2 END SMS
,CASE WHEN rpr.currency_id=1 THEN rpr.mt_sms_rate WHEN rpr.currency_id=2 THEN rpr.mt_sms_rate*1.2 END mt_sms_rate
,CASE WHEN rpr.currency_id=1 THEN rpr.data_rate WHEN rpr.currency_id=2 THEN rpr.data_rate*1.2 END MB
,'USD' Currency
,CASE WHEN rpr.sponsor_id=1 THEN 'S1' WHEN rpr.sponsor_id=2 THEN 'S2' END AS Sponsor
FROM ROAMING_PLAN_RULES rpr
LEFT JOIN RDB_NETWORKS n
ON rpr.network_id=n.network_id
LEFT JOIN RDB_COUNTRIES c
ON n.country_id=c.country_id
LEFT JOIN RDB_TADIG_CODES td
ON rpr.network_id=td.network_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mn
ON rpr.network_id=mn.network_id
WHERE rpr.roaming_plan_id=268 --P4
AND rpr.end_date IS NULL
AND rpr.is_discounted=1
'''
    
df_p4_cost = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

# display(df_p4_cost.head())

#Connect to RoamDB
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.19.11.59;DATABASE=RoamDB;UID=roamdb;PWD=roamdbpasswd')
#RoamDB P4 Cost (tarifflistid=27)
sql_srt=\
'''
SELECT t.outgoingvoiceincrement_SEC AS Voice_Increment_SEC
,t.dataincrement_KB AS Data_Increment_KB
,t.support4G,tc.value FROM Tariffs t
INNER JOIN TadigCodes tc
ON t.OperatorID = tc.OperatorID
WHERE t.tarifflistid = 27 --'P4 Mobile Costs 2019'
'''
df_p4_2019 = pd.read_sql_query(sql_srt, cnxn)
cnxn.close()

# display(df_p4_2019.head())

df_p4=pd.merge(df_p4_cost, df_p4_2019, how='left',left_on='TADIG',right_on='value')

display(df_p4.head(3))

,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,Voice_Increment_SEC,Data_Increment_KB,support4G,value
0,Albania,Vodafone,ALBVF,27602,0.0439,0.0,0.0062,0.0,0.0030,USD,S1,1.0,1.0,True,ALBVF
1,Andorra,Andorra Telecom S A U,ANDMA,21303,0.2000,0.0,0.0300,0.0,0.0500,USD,S1,60.0,1.0,False,ANDMA
2,Anguilla,Cable and Wireless,AIACW,365840,0.1606,0.0,0.0128,0.0,0.0257,USD,S1,60.0,1.0,False,AIACW


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,Voice_Increment_SEC,Data_Increment_KB,support4G,value
0,Albania,TELEKOM ALBANIA SH A,ALBAM,27601,0.5388,0.0,0.0624,0.0,0.0480,USD,S2,1.0,1.0,True,ALBAM
1,Albania,Eagle Mobile,ALBEM,27603,0.4680,0.0,0.0480,0.0,0.0468,USD,S2,60.0,1.0,True,ALBEM
2,Anguilla,Cable and Wireless,AIACW,365840,0.9600,0.0,0.0960,0.0,0.0960,USD,S2,1.0,1.0,False,AIACW


In [5]:
# Загружаем Allowed list с необходимыми изменениями

df_al=pd.read_csv(join(load_folder, file_al),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al,df_tadig[['TADIG_CODE','NETWORK_ID']],how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE', inplace=True,axis=1)

print('Allowed list:')
display(df_al.head(3))

Allowed list:


,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,S2,HRVT2,225.0
1,S2,MNEMT,622.0
2,S1,MKDCC,569.0


In [6]:
#Объединяем тарифы с TADIG и Allowed list (пока сделано только для Partner и P4)

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')

# Результирующий файл

f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2',
          'SPONSOR','TADIG_CODE_AL','NETWORK_ID']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head()

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,Voice_Increment_SEC,Data_Increment_KB,support4G,value,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,Operator,TADIG,MCCMNC,MOC,MTC,SMS,mt_sms_rate,MB,Currency,Sponsor,Voice_Increment_SEC,Data_Increment_KB,support4G,value,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.0500,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.96,0.0,0.096,0.096,1.0,1.0,S1,AIACW,20.0
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.96,0.0,0.096,0.096,1.0,1.0,S1,ATGCW,23.0
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0


In [7]:
# Финальная версия с MCCMNC

df_s1_s2_sponsor_mccmnc = pd.merge(df_s1_s2_sponsor, df_tadig_mccmnc, how='left',on='NETWORK_ID')
df_s1_s2_sponsor_mccmnc.head()

,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,Voice_Increment_SEC_S1,Data_Increment_KB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,Voice_Increment_SEC_S2,Data_Increment_KB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,MCCMNC
0,Albania,Vodafone,ALBVF,0.0439,0.0,0.0062,0.0030,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ALBVF,12.0,27602
1,Andorra,Andorra Telecom S A U,ANDMA,0.2000,0.0,0.0300,0.0500,60.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,21303
2,Anguilla,Cable and Wireless,AIACW,0.1606,0.0,0.0128,0.0257,60.0,1.0,0.96,0.0,0.096,0.096,1.0,1.0,S1,AIACW,20.0,365840
3,Antigua and Barbuda,C & W,ATGCW,0.1500,0.0,0.0120,0.0240,60.0,1.0,0.96,0.0,0.096,0.096,1.0,1.0,S1,ATGCW,23.0,344920
4,Argentina,Movistar/Telefonica,ARG01,0.0728,0.0,0.0240,0.0180,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,S1,ARG01,27.0,72207


In [8]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

df = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df[df.values > 1]
display(s1)

df = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df[df.values > 1]
display(s2)

,MOC,MTC,SMS,MB
NETWORK_ID,,,,


,MOC,MTC,SMS,MB
NETWORK_ID,,,,


In [8]:
# Подготовка Combined Price Plan с MCCMNC
# ИСПОЛЬЗОВАТЬ

df1 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR','NETWORK_ID']].copy()

df1.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df1['SPONSOR'] = 'S1'

df2 = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR','NETWORK_ID']].copy()

df2.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df2['SPONSOR'] = 'S2'

#df2[['MOC','MTC','SMS','DATA (MB)']] = df2[['MOC','MTC','SMS','DATA (MB)']]*eur_usd

df_combined = pd.concat([df1, df2],axis=0).drop_duplicates().sort_values(by=['Country','Operator'])

df_combined.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all',inplace=True)

df_combined.MTC.fillna(0,inplace=True)

df_combined[['MOC','MTC','SMS','DATA (MB)']] = df_combined[['MOC','MTC','SMS','DATA (MB)']]*2

df_combined.loc[df_combined.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_combined[df_combined.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_combined['CountrySponsor'] = df_combined.groupby('Country')['SPONSOR_AL'].transform('any')

df_combined = df_combined[df_combined.SPONSOR == df_combined.CountrySponsor]

df_combined.to_csv(join(load_folder,file_result),index=False)

df_combined.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR_AL,NETWORK_ID,SPONSOR,CountrySponsor
0,Albania,Vodafone,ALBVF,27602,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1,12.0,S1,S1
2,Anguilla,Cable and Wireless,AIACW,365840,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1,20.0,S1,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1,23.0,S1,S1
4,Argentina,Movistar/Telefonica,ARG01,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1,27.0,S1,S1
5,Argentina,Movistar/Telefonica,ARGTM,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1,27.0,S1,S1


In [26]:
# Подготовка Combined Price Plan (без MCCMNC)
# НЕ ИСПОЛЬЗОВАТЬ

df1 = df_s1_s2_sponsor\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR','NETWORK_ID']].copy()

df1.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df1['SPONSOR'] = 'S1'

df2 = df_s1_s2_sponsor\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR','NETWORK_ID']].copy()

df2.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df2['SPONSOR'] = 'S2'

#df2[['MOC','MTC','SMS','DATA (MB)']] = df2[['MOC','MTC','SMS','DATA (MB)']]*eur_usd

df_combined = pd.concat([df1, df2], axis=0).drop_duplicates().sort_values(by=['Country','Operator'])

df_combined.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all',inplace=True)

df_combined.MTC.fillna(0,inplace=True)

df_combined[['MOC','MTC','SMS','DATA (MB)']] = df_combined[['MOC','MTC','SMS','DATA (MB)']]*2

df_combined.loc[df_combined.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_combined[df_combined.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_combined['CountrySponsor'] = df_combined.groupby('Country')['SPONSOR_AL'].transform('any')

df_combined = df_combined[df_combined.SPONSOR == df_combined.CountrySponsor]

df_combined.to_csv(join(load_folder,file_result),index=False)

df_combined.head()

,Country,Operator,TADIG,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR_AL,NETWORK_ID,SPONSOR,CountrySponsor
0,Albania,Vodafone,ALBVF,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1,12.0,S1,S1
1,Andorra,Andorra Telecom S A U,ANDMA,0.4000,0.0,0.0600,0.1000,60.0,1.0,S1,17.0,S1,S1
2,Anguilla,Cable and Wireless,AIACW,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1,20.0,S1,S1
3,Antigua and Barbuda,C & W,ATGCW,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1,23.0,S1,S1
4,Argentina,Movistar/Telefonica,ARG01,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1,27.0,S1,S1


In [9]:
# Подготовка Partner Price Plan для Пользователей

df_s1_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S1','MTC_S1','SMS_S1','MB_S1','Voice_Increment_SEC_S1','Data_Increment_KB_S1','SPONSOR']].drop_duplicates()

df_s1_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S1':'MOC','MTC_S1':'MTC','SMS_S1':'SMS','MB_S1':'DATA (MB)',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)


df_s1_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s1_price_plan.MTC.fillna(0,inplace=True)

df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s1_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

df_s1_price_plan.loc[df_s1_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_s1_price_plan[df_s1_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s1_price_plan.to_csv(join(load_folder,'df_s1_price_plan.csv'),index=False)

df_s1_price_plan.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR
0,Albania,Vodafone,ALBVF,27602,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1
1,Andorra,Andorra Telecom S A U,ANDMA,21303,0.4000,0.0,0.0600,0.1000,60.0,1.0,NaN
2,Anguilla,Cable and Wireless,AIACW,365840,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1
4,Argentina,Movistar/Telefonica,ARG01,72207,0.1456,0.0,0.0480,0.0360,1.0,1.0,S1


In [10]:
df_s1_price_plan.count()

Country                  269
Operator                 269
TADIG                    269
MCCMNC                   269
MOC                      269
MTC                      269
SMS                      269
DATA (MB)                269
Voice Increment (SEC)    267
Data Increment (KB)      267
SPONSOR                  256
dtype: int64

In [11]:
# Подготовка P4 Price Plan для Пользователей

df_s2_price_plan = df_s1_s2_sponsor_mccmnc\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC_S2','MTC_S2','SMS_S2','MB_S2','Voice_Increment_SEC_S2','Data_Increment_KB_S2','SPONSOR']].drop_duplicates()

df_s2_price_plan.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC_S2':'MOC','MTC_S2':'MTC','SMS_S2':'SMS','MB_S2':'DATA (MB)',
            'Voice_Increment_SEC_S2':'Voice Increment (SEC)','Data_Increment_KB_S2':'Data Increment (KB)'}
           ,axis=1,inplace=True)

df_s2_price_plan.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all', inplace=True)
df_s2_price_plan.MTC.fillna(0,inplace=True)

df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']] = df_s2_price_plan[['MOC','MTC','SMS','DATA (MB)']]*2

df_s2_price_plan.loc[df_s2_price_plan.Country=='United States',['MOC','MTC','SMS','DATA (MB)']]\
= df_s2_price_plan[df_s2_price_plan.Country=='United States'][['MOC','MTC','SMS','DATA (MB)']]*1.25

df_s2_price_plan.to_csv(join(load_folder,'df_s2_price_plan.csv'),index=False)

df_s2_price_plan.head()

,Country,Operator,TADIG,MCCMNC,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR
2,Anguilla,Cable and Wireless,AIACW,365840,1.920,0.0,0.1920,0.1920,1.0,1.0,S1
3,Antigua and Barbuda,C & W,ATGCW,344920,1.920,0.0,0.1920,0.1920,1.0,1.0,S1
6,Armenia,ArmenTel,ARM01,28301,1.404,0.0,0.0360,0.0096,60.0,1.0,S1
9,Australia,Singtel Optus,AUSOP,50590,0.132,0.0,0.0192,0.0192,60.0,1.0,S1
10,Austria,A1 Telekom Austria AG,AUTMK,23211,0.072,0.0,0.0192,0.0084,1.0,1.0,S2


In [12]:
df_s2_price_plan.count()

Country                  225
Operator                 225
TADIG                    225
MCCMNC                   225
MOC                      225
MTC                      225
SMS                      225
DATA (MB)                225
Voice Increment (SEC)    225
Data Increment (KB)      225
SPONSOR                  184
dtype: int64

In [ ]:
# Подготовка MB Price Plan для Пользователей

# Добавляем MCCMNC из GSMA

df_mb_mccmnc = pd.merge(df_mb, df_tadig_gsma, how='left', on='TADIG')

# Добавляем название оператора и страны из OSC

df_mb_mccmnc_country = pd.merge(df_mb_mccmnc, df_tadig, how='left', left_on='TADIG', right_on='TADIG_CODE')

# Получаем итоговый файл

df4 = df_mb_mccmnc_country\
[['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MCCMNC'
  ,'MOC','MTC','SMS','MB','Voice_Increment_SEC_S1','Data_Increment_KB_S1','NETWORK_ID']].copy()

df4.rename({'COUNTRY_NAME':'Country','NETWORK_NAME':'Operator','TADIG_CODE':'TADIG',
            'MOC':'MOC','MTC':'MTC','SMS':'SMS','MB':'DATA (MB)','SPONSOR':'SPONSOR_AL',
            'Voice_Increment_SEC_S1':'Voice Increment (SEC)','Data_Increment_KB_S1':'Data Increment (KB)'}
           ,axis=1,inplace=True)
df4['SPONSOR'] = 'S4'

df_mb_price = df4.drop_duplicates()

df_mb_price.dropna(subset=['MOC','MTC','SMS','DATA (MB)'], how='all',inplace=True)

df_mb_price.MTC.fillna(0,inplace=True)

df_mb_price[['MOC','MTC','SMS','DATA (MB)']] = df_mb_price[['MOC','MTC','SMS','DATA (MB)']]*2

df_mb_price.to_csv(join(load_folder,file_result),index=False)

df_mb_price.head()

In [33]:
# Добавление MCCMNC GSMA

# Читаем данные из GSMA файла
df_tadig_gsma = pd.read_csv(join(load_folder, file_tagid_list_gsma), sep=';', dtype='object',
                       usecols=['TADIG','MCC','MNC'])
df_tadig_gsma['MCCMNC'] = df_tadig_gsma['MCC'] + df_tadig_gsma['MNC']
df_tadig_gsma.drop(['MCC','MNC'], axis=1, inplace=True)

# Добавить данные из IR21
tadig_mccmnc_ir21 = pd.read_csv(join(load_folder, file_tagid_list_mccmnc_ir21), dtype='object')
df_tadig_gsma = df_tadig_gsma.append(tadig_mccmnc_ir21[['TADIG','MCCMNC']])
df_tadig_gsma.drop_duplicates(inplace=True)

df_tadig_gsma.head(3)

,TADIG,MCCMNC
0,NZLNH,53024
2,IRLDF,27202
5,IRLH3,27205


In [34]:
df = df_combined.copy()
df.drop('MCCMNC', axis=1, inplace=True)

df_combined_mccmnc_gsma = pd.merge(df, df_tadig_gsma, how='left', on='TADIG').copy()
df_combined_mccmnc_gsma.drop_duplicates(inplace=True)

df_combined_mccmnc_gsma.to_csv(join(load_folder,file_result),index=False)

df_combined_mccmnc_gsma.head(3)

,Country,Operator,TADIG,MOC,MTC,SMS,DATA (MB),Voice Increment (SEC),Data Increment (KB),SPONSOR_AL,NETWORK_ID,SPONSOR,CountrySponsor,MCCMNC
0,Albania,Vodafone,ALBVF,0.0878,0.0,0.0124,0.0060,1.0,1.0,S1,12.0,S1,S1,27602
1,Anguilla,Cable and Wireless,AIACW,0.3212,0.0,0.0256,0.0514,60.0,1.0,S1,20.0,S1,S1,365840
2,Antigua and Barbuda,C & W,ATGCW,0.3000,0.0,0.0240,0.0480,60.0,1.0,S1,23.0,S1,S1,344920


In [13]:
# Подготовка файла тарифов для загрузки в базу данных OCS для Real Combined Price Plan

# ??? Не понял ???

# Файлы:
file_combined_100 = 'df_combined_100_with_new.csv'

df_combined_100 = df_combined[df_combined.SPONSOR_AL.isnull()].copy()
df_combined_100.rename({'DATA (MB)':'DATA', 'SPONSOR':'SPONSOR_ID'},axis=1,inplace=True)
df_combined_100.SPONSOR_ID.replace(to_replace={'S1':1, 'S2':2}, inplace = True)

#Добавляем поля для удобства выгрузки
#!!! Поменять 'ROAMING_PLAN_ID', 'CURRENCY_ID', 'SPONSOR_ID' и 'START_DATE'

df_combined_100['ROAMING_PLAN_RULE_ID']=''
df_combined_100['ROAMING_PLAN_ID']=''
df_combined_100['MT_SMS_RATE']=0
df_combined_100['CURRENCY_ID']=1 #1:USD, 2:EUR
df_combined_100['START_DATE']='2019-01-01 00:00:00'
df_combined_100['END_DATE']='NULL'
df_combined_100['is_discounted']=1

df_combined_100[['TADIG',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC',
'MTC',
'SMS',
'MT_SMS_RATE',
'DATA',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted',]].drop_duplicates().to_csv(join(load_folder,file_combined_100),index=False)